In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd

In [2]:
# A empresa e Indicadores fundamentalistas dados do statusinvest.com.br
#(pode ter atraso, dados não são pegos diretamente no SI)

url = 'https://raw.githubusercontent.com/BDonadelli/Codigos-em-financas/main/data/SI_Acoes.csv'
pd.set_option('display.max_columns', None)
fundsSI = pd.read_csv(url,sep=';' , decimal=',' ,thousands ='.' )

In [3]:
fundsSI.columns

Index(['TICKER', 'PRECO', 'DY', 'P/L', 'P/VP', 'P/ATIVOS', 'MARGEM BRUTA',
       'MARGEM EBIT', 'MARG. LIQUIDA', 'P/EBIT', 'EV/EBIT',
       'DIVIDA LIQUIDA / EBIT', 'DIV. LIQ. / PATRI.', 'PSR', 'P/CAP. GIRO',
       'P. AT CIR. LIQ.', 'LIQ. CORRENTE', 'ROE', 'ROA', 'ROIC',
       'PATRIMONIO / ATIVOS', 'PASSIVOS / ATIVOS', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', ' LIQUIDEZ MEDIA DIARIA',
       ' VPA', ' LPA', ' PEG Ratio', ' VALOR DE MERCADO'],
      dtype='object')

In [4]:
fundsSI =  fundsSI[fundsSI[' LIQUIDEZ MEDIA DIARIA'] > 250000] 
fundsSI =  fundsSI[(fundsSI[' LPA'] > 0) & (fundsSI[' VPA'] > 0) ]
# fundsSI

In [5]:
fundsSI['valor_intrinseco'] = np.sqrt(22.5 * fundsSI[' LPA'] * fundsSI[' VPA'])
fundsSI['Delta'] = fundsSI['valor_intrinseco'] / fundsSI['PRECO'] -1
fundsSI["Rank"]   = fundsSI['Delta'].rank(ascending=True, method="min")
fundsSI.sort_values(by="Rank", ascending=False, inplace=True)
fundsSI.reset_index(inplace=True)
fundsSI.index = fundsSI.index + 1

top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,1.78,8.703749,199.0,3.889747,0.87,3.87
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
3,GOAU4,10.84,32.252023,197.0,1.975279,2.66,17.38
4,GOAU3,10.98,32.252023,196.0,1.937343,2.66,17.38
5,HBOR3,2.93,8.386596,195.0,1.862320,0.30,10.42
6,COCE5,35.08,92.970298,194.0,1.650237,7.05,54.49
7,GGBR3,20.79,54.774447,193.0,1.634654,4.63,28.80
8,BRSR6,13.47,34.915348,192.0,1.592082,2.29,23.66
9,MTRE3,5.11,13.056722,191.0,1.555132,0.82,9.24
10,HBRE3,5.98,14.570089,190.0,1.436470,0.51,18.50


In [6]:
fundsSI =  fundsSI[(fundsSI['P/L'] > 0)]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
1,LUPA3,1.78,8.703749,199.0,3.889747,0.87,3.87
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
3,GOAU4,10.84,32.252023,197.0,1.975279,2.66,17.38
4,GOAU3,10.98,32.252023,196.0,1.937343,2.66,17.38
5,HBOR3,2.93,8.386596,195.0,1.862320,0.30,10.42
6,COCE5,35.08,92.970298,194.0,1.650237,7.05,54.49
7,GGBR3,20.79,54.774447,193.0,1.634654,4.63,28.80
8,BRSR6,13.47,34.915348,192.0,1.592082,2.29,23.66
9,MTRE3,5.11,13.056722,191.0,1.555132,0.82,9.24
10,HBRE3,5.98,14.570089,190.0,1.436470,0.51,18.50


In [7]:
fundsSI =  fundsSI[(fundsSI['CAGR RECEITAS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
3,GOAU4,10.84,32.252023,197.0,1.975279,2.66,17.38
4,GOAU3,10.98,32.252023,196.0,1.937343,2.66,17.38
5,HBOR3,2.93,8.386596,195.0,1.862320,0.30,10.42
6,COCE5,35.08,92.970298,194.0,1.650237,7.05,54.49
7,GGBR3,20.79,54.774447,193.0,1.634654,4.63,28.80
8,BRSR6,13.47,34.915348,192.0,1.592082,2.29,23.66
9,MTRE3,5.11,13.056722,191.0,1.555132,0.82,9.24
10,HBRE3,5.98,14.570089,190.0,1.436470,0.51,18.50
12,SAPR3,4.94,11.827383,188.0,1.394207,0.99,6.28


In [8]:
fundsSI =  fundsSI[(fundsSI['CAGR LUCROS 5 ANOS'] > 0) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
6,COCE5,35.08,92.970298,194.0,1.650237,7.05,54.49
12,SAPR3,4.94,11.827383,188.0,1.394207,0.99,6.28
14,EUCA4,16.95,39.888636,186.0,1.353312,2.97,23.81
16,ALOS3,24.03,56.101845,184.0,1.334659,5.46,25.62
18,NEOE3,20.78,48.354565,182.0,1.326976,4.28,24.28
19,SAPR11,25.70,59.036120,181.0,1.297125,4.93,31.42
20,SAPR4,5.17,11.827383,180.0,1.287695,0.99,6.28
21,BMGB4,3.46,7.791341,179.0,1.251833,0.38,7.10
22,NTCO3,17.84,39.493123,178.0,1.213740,3.43,20.21


In [9]:
fundsSI =  fundsSI[(fundsSI['DIVIDA LIQUIDA / EBIT'] <4) ]
top20 = fundsSI[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)
top20

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
12,SAPR3,4.94,11.827383,188.0,1.394207,0.99,6.28
14,EUCA4,16.95,39.888636,186.0,1.353312,2.97,23.81
16,ALOS3,24.03,56.101845,184.0,1.334659,5.46,25.62
18,NEOE3,20.78,48.354565,182.0,1.326976,4.28,24.28
19,SAPR11,25.70,59.036120,181.0,1.297125,4.93,31.42
20,SAPR4,5.17,11.827383,180.0,1.287695,0.99,6.28
22,NTCO3,17.84,39.493123,178.0,1.213740,3.43,20.21
28,CMIG4,13.00,27.554310,172.0,1.119562,3.04,11.10
30,ROMI3,12.38,26.111765,170.0,1.109189,2.49,12.17


In [10]:
fundsSI['prefixo'] = fundsSI['TICKER'].str[:4]
fundsSI.drop_duplicates(subset='prefixo', keep='first').head(20)
df_limpo = fundsSI.drop_duplicates(subset='prefixo', keep='first')
df_limpo = df_limpo.drop('prefixo', axis=1) 
df_limpo[['TICKER','PRECO' , 'valor_intrinseco' , 'Rank' , 'Delta' , ' LPA' , ' VPA' ]].head(20)

,TICKER,PRECO,valor_intrinseco,Rank,Delta,LPA,VPA
2,MEAL3,1.65,5.982725,198.0,2.625894,0.41,3.88
12,SAPR3,4.94,11.827383,188.0,1.394207,0.99,6.28
14,EUCA4,16.95,39.888636,186.0,1.353312,2.97,23.81
16,ALOS3,24.03,56.101845,184.0,1.334659,5.46,25.62
18,NEOE3,20.78,48.354565,182.0,1.326976,4.28,24.28
22,NTCO3,17.84,39.493123,178.0,1.213740,3.43,20.21
28,CMIG4,13.00,27.554310,172.0,1.119562,3.04,11.10
30,ROMI3,12.38,26.111765,170.0,1.109189,2.49,12.17
34,CAML3,8.49,16.312526,166.0,0.921381,1.38,8.57
36,FESA4,8.80,16.778558,164.0,0.906654,1.36,9.20


In [11]:
# linhas_com_nan = top20[top20.isna().any(axis=1)]
# linhas_com_nan

### Porque 22,5?

O número 22,5 na Fórmula de Graham é um fator de ponderação que tem um propósito específico. Essa constante foi escolhida por Graham pra ajustar a avaliação do preço justo de uma ação com base na taxa de crescimento anual esperada da empresa.

O número 22,5 é, na verdade, o resultado da multiplicação de 8,5 por 2,65 (8,5 x 2,65 = 22,5). O número 8,5 é a base que Graham considerou razoável pra uma empresa com taxa de crescimento zero, ou seja, uma empresa que não cresce.

Já o número 2,65 representa a média do retorno exigido pelos investidores no mercado de ações durante a época de Graham, que era de aproximadamente 4,4% acima da taxa de retorno dos títulos do Tesouro dos Estados Unidos.

O fator 22,5 ajuda a ajustar o preço justo com base no crescimento da empresa e na expectativa de retorno dos investidores.

Esse ajuste garante que a Fórmula de Graham considere a taxa de crescimento anual esperada e reflita uma avaliação mais realista do preço justo de uma ação.

No próximo tópico, abordaremos a calculadora do preço justo usando a Fórmula de Graham.